## Connect to client and view channel info

In [1]:
from telethon import TelegramClient
from telethon.tl.functions.channels import GetFullChannelRequest, JoinChannelRequest
import configparser
import pandas as pd

# Setting config values
config = configparser.ConfigParser()
config.read("/Users/dankoban/Documents/Dissertation/config.ini")
api_id = config['telegram']['api_id']
api_hash = config['telegram']['api_hash']
phone = config['telegram']['phone']
bot_token = config['telegram']['bot_token']

In [2]:
client = TelegramClient('user', api_id, api_hash)
await client.connect()
await client.send_code_request(f"+{phone2}", force_sms=True)
code = input('enter code: ')
await client.sign_in(phone, code)
await client.get_participants('https://t.me/learningtelegram2')

enter code: 48819


[<telethon.tl.types.User object at 0x7fba5834ed90>, <telethon.tl.types.User object at 0x7fba5834e5b0>, <telethon.tl.types.User object at 0x7fba5834eb20>, total=3]

In [3]:
r = await client(GetFullChannelRequest(channel='https://t.me/learningtelegram2'))
ch_full = pd.json_normalize(r.to_dict())    
ch_full['title'] = pd.json_normalize(r.to_dict())['chats'][0][0]['title']
ch_full = ch_full[['full_chat.id', 
                   'title',                       
                   'full_chat.about', 
                   'full_chat.can_view_participants', 
                   'full_chat.participants_count']]
ch_full

,full_chat.id,title,full_chat.about,full_chat.can_view_participants,full_chat.participants_count
0,1543257888,Learning Telegram channel,Learning how to use Telegram with Python,True,3


In [4]:
members = await client.get_participants('https://t.me/learningtelegram2')
member_list = []
for i in range(0, len(members)):
    try:
        member_list.append(pd.DataFrame({'id': [members[i].to_dict()['id']],
                                         'username': [members[i].to_dict()['username']],
                                         'bot': [members[i].to_dict()['bot']],
                                         'first_name': [members[i].to_dict()['first_name']],
                                         'last_name': [members[i].to_dict()['last_name']]}))
    except:
        pass
member_df = pd.concat(member_list)
member_df.reset_index(inplace = True, drop = True)
member_df

,id,username,bot,first_name,last_name
0,1889625564,RandomPerson4,False,Random,Person
1,1790813816,survey_014014014_bot,True,survey_bot_14,None
2,1818726722,RandomPerson5,False,Random,Person5


## Send a message via API

In [8]:
await client.send_message('https://t.me/learningtelegram2', 'I sent the google webpage via Python telethon https://www.google.com/')

In [10]:
chats = await client.get_messages('https://t.me/learningtelegram2', None)
print(chats[0])

Message(id=58, peer_id=PeerChannel(channel_id=1543257888), date=datetime.datetime(2021, 8, 23, 14, 30, 31, tzinfo=datetime.timezone.utc), message='I sent the google webpage via Python telethon https://www.google.com/', out=True, mentioned=False, media_unread=False, silent=False, post=True, from_scheduled=False, legacy=False, edit_hide=False, pinned=False, from_id=None, fwd_from=None, via_bot_id=None, reply_to=None, media=None, reply_markup=None, entities=[MessageEntityUrl(offset=46, length=23)], views=2, forwards=0, replies=MessageReplies(replies=0, replies_pts=140, comments=True, recent_repliers=[], channel_id=1507922499, max_id=None, read_max_id=None), edit_date=None, post_author='Random Person5', grouped_id=None, restriction_reason=[], ttl_period=None)


In [11]:
chats = await client.get_messages('https://t.me/learningtelegram2', None)
await chats[0].reply("and posted a follow on comment")

In [12]:
bot_client = TelegramClient('bot', api_id, api_hash)
await bot_client.start(bot_token = bot_token)
await bot_client.connect()

In [13]:
chats = await bot_client.get_messages('https://t.me/learningtelegram2', None)
await chats[0].reply("beep beep boop...I cannot post replies as a bot")

BotMethodInvalidError: The API access for bot users is restricted. The method you tried to invoke cannot be executed as a bot (caused by GetHistoryRequest)

In [14]:
await bot_client.send_message('https://t.me/learningtelegram2', 'Beep Beep Boop...I am a bot')

In [17]:
await bot_client.send_message(entity = 'https://t.me/learningtelegram2', 
                              message = 'replying from bot',
                              reply_to = 58)

#await bot_client.send_message(entity = 'https://t.me/learningtelegram2', 
#                              message = 'commenting on a post from bot',
#                              comment_to = 58)

In [26]:
import asyncio
from telethon import events

@bot_client.on(events.NewMessage(pattern='(?i)hello.+'))
async def handler(event):
    # Respond whenever someone says "Hello" and something else
    await event.reply('Hey!')

@bot_client.on(events.NewMessage(outgoing=True, pattern='!ping'))
async def handler(event):
    # Say "!pong" whenever you send "!ping", then delete both messages
    #m = await event.respond('!pong')
    m = await event.reply('!pong')
    #await asyncio.sleep(5)
    #await client.delete_messages(event.chat_id, [event.id, m.id])

In [20]:
chats = await client.get_messages('https://t.me/learningtelegram2', None)
print(chats[1])

Message(id=60, peer_id=PeerChannel(channel_id=1543257888), date=datetime.datetime(2021, 8, 23, 14, 32, 2, tzinfo=datetime.timezone.utc), message='Beep Beep Boop...I am a bot', out=False, mentioned=False, media_unread=False, silent=False, post=True, from_scheduled=False, legacy=False, edit_hide=False, pinned=False, from_id=None, fwd_from=None, via_bot_id=None, reply_to=None, media=None, reply_markup=None, entities=[], views=2, forwards=0, replies=MessageReplies(replies=0, replies_pts=146, comments=True, recent_repliers=[], channel_id=1507922499, max_id=None, read_max_id=None), edit_date=None, post_author='survey_bot_14', grouped_id=None, restriction_reason=[], ttl_period=None)


In [21]:
await client.send_message(entity = 'https://t.me/learningtelegram2', 
                          message = 'hello bot',
                          reply_to = 60)

In [22]:
await client.send_message(entity = 'https://t.me/learningtelegram2', 
                          message = '!ping',
                          reply_to = 60)

In [27]:
await client.send_message('https://t.me/learningtelegram2', '!ping')

In [28]:
await bot_client.send_message('https://t.me/learningtelegram2', '!ping')

In [25]:
await client.send_message('https://t.me/learningtelegram2', 'hello bot')

## Query channel for comments

In [56]:
r = await client(GetFullChannelRequest(channel='https://t.me/TrumpChannel'))
ch_full = pd.json_normalize(r.to_dict())    
ch_full['title'] = pd.json_normalize(r.to_dict())['chats'][0][0]['title']
ch_full = ch_full[['full_chat.id', 
                   'title',                       
                   'full_chat.about', 
                   'full_chat.can_view_participants', 
                   'full_chat.participants_count']]
ch_full

,full_chat.id,title,full_chat.about,full_chat.can_view_participants,full_chat.participants_count
0,1459890679,Trump Supporters Channel 🇺🇸,Supporters of the greatest President of our li...,False,401054


In [58]:
chats = await client.get_messages('https://t.me/TrumpChannel', 1)
print("message ID: " + str(chats[0].id))
print(chats[0].message)
print("comment count:" + str(chats[0].replies.replies))
print("last replier: " + str(chats[0].replies.recent_repliers[0].to_dict()['user_id']))

message ID: 1989
😳

Via @WesternJournal
comment count:182
last replier: 1350480243


In [61]:
comments = await client.get_messages('https://t.me/TrumpChannel', 1, reply_to = chats[0].id)
comments[0].to_dict()

{'_': 'Message',
 'id': 820051,
 'peer_id': {'_': 'PeerChannel', 'channel_id': 1332262008},
 'date': datetime.datetime(2021, 8, 23, 14, 59, 32, tzinfo=datetime.timezone.utc),
 'message': 'Trump did have a date to get our people home.\nBut he HAD A PLAN!! Dumbass BIDEN, dismissed it because TRUMP HAD HIS OWN PLAN.\nLike everything else good Trump did and promised us and delivered!!!!\nIf some of you STILL DON’T SEE AND UNDERSTAND WHAT IS ACTUALLY GOING ON, then there is no help for you.',
 'out': False,
 'mentioned': False,
 'media_unread': False,
 'silent': False,
 'post': False,
 'from_scheduled': False,
 'legacy': False,
 'edit_hide': False,
 'pinned': False,
 'from_id': {'_': 'PeerUser', 'user_id': 1655364946},
 'fwd_from': None,
 'via_bot_id': None,
 'reply_to': {'_': 'MessageReplyHeader',
  'reply_to_msg_id': 819204,
  'reply_to_peer_id': None,
  'reply_to_top_id': None},
 'media': None,
 'reply_markup': None,
 'entities': [],
 'views': None,
 'forwards': None,
 'replies': None,
 

In [62]:
await client.log_out()

True

In [63]:
await bot_client.log_out()

True